In [ ]:
# default_exp diff_analysis_manager

In [ ]:
from ms_empire.background_distributions import *
from ms_empire.normalization import *
from ms_empire.diff_analysis import *

In [ ]:
#export
#read in peptide datafiles



In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np

def run_pipeline(unnormed_df, labelmap_df, minrep):
    conds = labelmap_df["condition"].unique()
    condpair2diffions = {}
    for condpair in combinations(conds, 2):

        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=c1_samples.shape[0]-minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=c2_samples.shape[0]-minrep, axis=0)

        df_c1_normed = pd.DataFrame(normalize_withincond(df_c1.to_numpy().T).T, index = df_c1.index)
        df_c2_normed = pd.DataFrame(normalize_withincond(df_c2.to_numpy().T).T, index = df_c2.index)

        normed_c1 = ConditionBackgrounds(df_c1_normed)
        normed_c2 = ConditionBackgrounds(df_c2_normed)

        ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()

        ion2diffDist = {}
        diffIons = []
        for ion in ions_to_check:
            vals1 = normed_c1.ion2nonNanvals.get(ion)
            vals2 = normed_c2.ion2nonNanvals.get(ion)
            bgdist1 = normed_c1.ion2background.get(ion)
            bgdist2 = normed_c2.ion2background.get(ion)
            diffDist = get_subtracted_bg(ion2diffDist,normed_c1, normed_c2,ion )
            diffIon = DifferentialIon(vals1, vals2, diffDist)
            diffIons.append(diffIon)

        condpair2diffions.update({condpair :diffIon})

In [ ]:
#hide
import pandas as pd
import numpy as np
import uuid

def test_run_pipeline():

    sample2cond_df = pd.DataFrame({'sample' : ['A1', 'A2', 'A3', 'B1', 'B2', 'B3'], 
    'condition' : ['A', 'A', 'A', 'B', 'B', 'B']})
    unnormed_df = generate_random_input(1000, sample2cond_df)
    
    run_pipeline(unnormed_df, sample2cond_df, 3)


def generate_random_input(num_pep,sample2cond_df ):
    pepnames = list(map(lambda _idx : str(uuid.uuid4()), range(num_pep))) #gives uuid strings for each peptide
    randarrays = 10+ 1.5*np.random.randn(num_pep,sample2cond_df.shape[0])
    df_intens = pd.DataFrame(randarrays, columns= sample2cond_df["sample"].tolist())
    df_intens.insert(0, "peptides", pepnames )
    df_intens = df_intens.set_index("peptides")
    return df_intens

test_run_pipeline()


ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, '9127b131-23bd-4934-b5a6-ee934da7b182': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, 'a817c712-9eeb-40ff-9517-4a4f84685b4b': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, '6c3920c8-facc-410c-b9cb-5ebbae2d48aa': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, '9a1f16fe-2fb7-4df5-ba61-dac2006b9efb': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, 'd23de24f-49e8-4216-9293-c776fbaffd2a': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, 'f1c8d3d8-5368-40f6-9ca2-d99f5b199779': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, '2fc22f7e-817a-47fb-9318-006b937a0c99': <ms_empire.background_distributions.SubtractedBackgrounds object at 0x1a22f7f150>, '85f5cea7-a157-445e-9aaa-680be8404bcd': <ms_empire.backgr

AttributeError: 'NoneType' object has no attribute 'start_idx'